In [52]:
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import json
import pymongo

In [2]:
url = "https://documentation.softwareag.com/webmethods/api_gateway/yai10-15/webhelp/yai-webhelp/yai-webhelp.html"
response = requests.request("GET", url, data="")

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

In [7]:
# write a function to get links in the deepest level of the soup
def get_links(soup):
    links = set()
    if soup.find('li') is None:
        links.add(soup.find('a').get('href'))
    else:
        for li in soup.find_all('li'):
            links = links.union(get_links(li))
    return links

In [9]:
allLinks = get_links(soup)

In [17]:
baseUrl = 'https://documentation.softwareag.com/webmethods/api_gateway/yai10-15/webhelp/yai-webhelp/'


In [18]:
links = [baseUrl + link for link in allLinks]

In [20]:
open('links.txt', 'w').write('\n'.join(links))

88503

In [22]:
driver = webdriver.Chrome("./chromedriver")

In [53]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
collection = client["documentation"]["yai"]

In [54]:
dataDict = {}
exceptionsList = []
for link in tqdm(links):
    try:
        driver.get(link)
        driver.implicitly_wait(30)
        title = driver.title
        driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        soup = soup.find('body')
        soup.header.decompose()
        soup.footer.decompose()
        data = soup.text.strip().replace('\n', ' ')
        temp = {'text': data, 'link': link, 'title': title}
        collection.insert_one(temp)
    except Exception as e:
        print(e)
        exceptionsList.append(link)

100%|██████████| 660/660 [02:54<00:00,  3.78it/s]


In [50]:

json.dump(dataDict, open('dataNew.json', 'w'))

In [62]:
"/".join(link.split('/')[:-2])

'https://documentation.softwareag.com/webmethods/api_gateway/yai10-15/webhelp/yai-webhelp'

In [58]:
baseUrl

'https://documentation.softwareag.com/webmethods/api_gateway/yai10-15/webhelp/yai-webhelp/'

In [68]:
url.split('-')[-2].split('/')[-1]

'yai'